In [1]:
from __future__ import print_function

from time import time
import os
import glob
import numpy as np
from skimage.transform import resize
from skimage.io import imsave
from PIL import Image
from skimage.io import imread
from skimage.transform import resize
from skimage.io import imsave
import matplotlib as mp
from matplotlib.colors import Normalize
import cv2
name='mask'
 
data_path = ''

image_rows = int(512)
image_cols = int(512)
image_depth = 8
if(name=='train'):
    img_input=os.path.join(data_path, 'Tac/')
    img_output=os.path.join(data_path, 'Train_f/')
if(name=='mask'):
    img_input=os.path.join(data_path, 'BMask/')
    img_output=os.path.join(data_path, 'Mask_f/')
if(name=='test'):
    img_input=os.path.join(data_path, 'Test/')
    img_output=os.path.join(data_path, 'Test_f/')
if(name=='val'):
    img_input=os.path.join(data_path, 'Val/')
    img_output=os.path.join(data_path, 'Val_f/')

In [2]:
tot=0
images = sorted(os.listdir(img_input))
for image_name in images:
    print(image_name,end="                     \r")
    img = imread(os.path.join(img_input, image_name), as_grey=True)
    tot+=len(img)
    #print(img.shape)
tot_batch=tot//image_depth
print('Total Batch')
print(str(tot_batch))

if(name=='mask'or (name=='val')):
    imgs = np.ndarray((tot_batch, image_depth, image_rows, image_cols), dtype=np.uint8)#Mask
    imgs_mask = np.ndarray((tot_batch, image_depth, image_rows, image_cols), dtype=np.uint8)   
if((name=='train')or(name=='test')):
    imgs = np.ndarray((tot_batch, image_depth, image_rows, image_cols), dtype=np.float32)#Immagini
    imgs_mask = np.ndarray((tot_batch, image_depth, image_rows, image_cols), dtype=np.float32) 


Total Batch                         
1504


In [3]:
i=0
remain=0

for image_name in images:
    start=time()
    k=0
    if (i==tot_batch-1):
        break
    print(image_name + " Time  " + str(remain),end="                                            \r")
    path=os.path.join(img_input, image_name)
    img = imread(path, as_grey=True)
    if((name=='train')or(name=='test')):
        img[np.where(img<-150)]=-150#IMG
        img[np.where(img>2200)]=2200#IMG
        img = cv2.normalize(img, None, norm_type=cv2.NORM_MINMAX)
        #print(img.shape)
    if(name=='mask'or(name=='val')):
        #img=resize(img, (len(img),512,512),anti_aliasing=True) #per le mask
        img[np.where(img>0)]=255  
        #print(img.shape,end="\r")
    for j in range(0 , len(img)):
        #print(k)#Debug
        if(k+image_depth>len(img)):
            break
        if(k+image_depth<=len(img)):
            for z in range (0, image_depth):
                if((name=='train')or(name=='test')):
                    imgs[i,z]=(img[k,:,:])
                if(name=='mask'or(name=='val')):  
                    imgs[i,z]=resize(img[k,:,:],(512,512),anti_aliasing=False) #per le mask
                    imgs[i,z,np.where(imgs[i,z]>0)]=1
                #print(imgs[i,z])
                k+=1
                if(z==image_depth-1):
                    i+=1
    end=time()
    remain=np.around((end-start),decimals=1)
for i in range (0,tot_batch):    
    if(name=='train'):
        out=os.path.join(img_output, str(i)+'.npy')
        np.save(out, np.expand_dims(imgs[i],axis=0))
    if(name=='mask'):
        out=os.path.join(img_output, str(i)+'.npy')
        np.save(out, np.expand_dims(imgs[i],axis=0))
    if(name=='test'):
        out=os.path.join(img_output, str(i)+'.npy')
        np.save(out, np.expand_dims(imgs[i],axis=0))
    if(name=='val'):
        out=os.path.join(img_output, str(i)+'.npy')
        np.save(out, np.expand_dims(imgs[i],axis=0))
print("File Saved")

File Savedd Time  7.6                                                
